## Imports


In [ ]:
%%capture
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

import torch
import torch.nn as nn
import numpy as np
import typing
from typing import Optional, Tuple, Union, Callable
import torch.nn.functional as F
import torch_geometric.utils as U
import torch_geometric
from torch_geometric.loader import DataLoader
import torch_geometric.datasets as datasets

from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import GATv2Conv
from torch_geometric.nn import GINConv
from torch_geometric.nn import Linear

from torch import Tensor
from torch.nn import Parameter
from torch_sparse import SparseTensor, set_diag

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import glorot, zeros, reset
from torch_geometric.typing import Adj, OptTensor, PairTensor
from torch_geometric.utils import add_self_loops, remove_self_loops, softmax

from torch_geometric.nn import global_mean_pool, global_add_pool

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Datasets


In [ ]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='.', name='IMDB-BINARY', transform=torch_geometric.transforms.OneHotDegree(138)).shuffle()

# Print information about the dataset
print(f'Dataset: {dataset}')

In [ ]:
# Divide dataset and create mini-batches
train_dataset = dataset[:int(len(dataset)*0.8)]
val_dataset   = dataset[int(len(dataset)*0.8):int(len(dataset)*0.9)]
test_dataset  = dataset[int(len(dataset)*0.9):]

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f'Training set: {len(train_dataset)} graphs')
print(f'Validation set: {len(val_dataset)} graphs')
print(f'Test set: {len(test_dataset)} graphs')

## Attention Graph Isomorphism Layer (AGINConv)

In [ ]:
class AGINConv(MessagePassing):
    r"""The AGIN layer from the `"Attention, here comes expressivity"` study, which combines the original 
    definition of the
    :class:`~torch_geometric.conv.GATv2Conv` layer with a learnable update function from
    :class:`~torch_geometric.conv.GINConv`.

    Args:
        in_channels (int): Size of each input sample, or :obj:`-1` to
            derive the size from the first input(s) to the forward method.
            A tuple corresponds to the sizes of source and target
            dimensionalities.
        out_channels (int): Size of each output sample.
        nn (torch.nn.Module): A neural network :math:`h_{\mathbf{\Theta}}` that 
            maps node features :obj:`x` of shape :obj:`[-1, in_channels]` to 
            shape :obj:`[-1, out_channels]`, *e.g.*, defined by 
            :class:`torch.nn.Sequential`.
        heads (int, optional): Number of multi-head-attentions.
            (default: :obj:`1`)
        concat (bool, optional): If set to :obj:`False`, the multi-head
            attentions are averaged instead of concatenated.
            (default: :obj:`True`)
        eps (float, optional): (Initial) :math:`\epsilon`-value.
            (default: :obj:`0.`)
        train_eps (bool, optional): If set to :obj:`True`, :math:`\epsilon`
            will be a trainable parameter. (default: :obj:`False`)
        negative_slope (float, optional): LeakyReLU angle of the negative
            slope. (default: :obj:`0.2`)
        dropout (float, optional): Dropout probability of the normalized
            attention coefficients which exposes each node to a stochastically
            sampled neighborhood during training. (default: :obj:`0`)
        edge_dim (int, optional): Edge feature dimensionality (in case
            there are any). (default: :obj:`None`)
        fill_value (float or Tensor or str, optional): The way to generate
            edge features of self-loops (in case :obj:`edge_dim != None`).
            If given as :obj:`float` or :class:`torch.Tensor`, edge features of
            self-loops will be directly given by :obj:`fill_value`.
            If given as :obj:`str`, edge features of self-loops are computed by
            aggregating all features of edges that point to the specific node,
            according to a reduce operation. (:obj:`"add"`, :obj:`"mean"`,
            :obj:`"min"`, :obj:`"max"`, :obj:`"mul"`). (default: :obj:`"mean"`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.

    Shapes:
        - **input:**
          node features :math:`(|\mathcal{V}|, F_{in})` or
          :math:`((|\mathcal{V_s}|, F_{s}), (|\mathcal{V_t}|, F_{t}))`
          if bipartite,
          edge indices :math:`(2, |\mathcal{E}|)`,
          edge features :math:`(|\mathcal{E}|, D)` *(optional)*
        - **output:** node features :math:`(|\mathcal{V}|, H * F_{out})` or
          :math:`((|\mathcal{V}_t|, H * F_{out})` if bipartite.
          If :obj:`return_attention_weights=True`, then
          :math:`((|\mathcal{V}|, H * F_{out}),
          ((2, |\mathcal{E}|), (|\mathcal{E}|, H)))`
          or :math:`((|\mathcal{V_t}|, H * F_{out}), ((2, |\mathcal{E}|),
          (|\mathcal{E}|, H)))` if bipartite
    """
    _alpha: OptTensor

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        nn: Callable,
        heads: int = 1,
        concat: bool = True,
        eps: float = 0.0,
        train_eps: bool = False,
        negative_slope: float = 0.2,
        dropout: float = 0.0,
        edge_dim: Optional[int] = None,
        fill_value: Union[float, Tensor, str] = 'mean',
        **kwargs,
    ):
        super().__init__(node_dim=0, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = dropout
        self.add_self_loops = True
        self.edge_dim = edge_dim
        self.fill_value = fill_value
        self.nn = nn
        self.initial_eps = eps

        if train_eps:
            self.eps = torch.nn.Parameter(torch.Tensor([eps]))
        else:
            self.register_buffer('eps', torch.Tensor([eps]))

        if isinstance(in_channels, int):
            self.lin = torch.cat([torch.eye(in_channels) for i in range(heads)], 0)
        else:
            raise NotImplementedError()

        self.att = Parameter(torch.Tensor(1, heads, in_channels))

        if edge_dim is not None:
            self.lin_edge = Linear(edge_dim, heads * out_channels, bias=False,
                                   weight_initializer='glorot')
        else:
            self.lin_edge = None

        self._alpha = None

        self.reset_parameters()

    def reset_parameters(self):
        if self.lin_edge is not None:
            self.lin_edge.reset_parameters()
        glorot(self.att)
        reset(self.nn)
        self.eps.data.fill_(self.initial_eps)

    def forward(self, x: Union[Tensor, PairTensor], edge_index: Adj,
                edge_attr: OptTensor = None,
                return_attention_weights: bool = None):
        # type: (Union[Tensor, PairTensor], Tensor, OptTensor, NoneType) -> Tensor  # noqa
        # type: (Union[Tensor, PairTensor], SparseTensor, OptTensor, NoneType) -> Tensor  # noqa
        # type: (Union[Tensor, PairTensor], Tensor, OptTensor, bool) -> Tuple[Tensor, Tuple[Tensor, Tensor]]  # noqa
        # type: (Union[Tensor, PairTensor], SparseTensor, OptTensor, bool) -> Tuple[Tensor, SparseTensor]  # noqa
        r"""
        Args:
            return_attention_weights (bool, optional): If set to :obj:`True`,
                will additionally return the tuple
                :obj:`(edge_index, attention_weights)`, holding the computed
                attention weights for each edge. (default: :obj:`None`)
        """
        H, C = self.heads, self.in_channels

        x_l: OptTensor = None
        x_r: OptTensor = None
        if isinstance(x, Tensor):
            assert x.dim() == 2
            x_l = torch.nn.functional.linear(x, self.lin).view(-1, H, C)
            x_r = x_l

        assert x_l is not None
        assert x_r is not None

        if self.add_self_loops:
            if isinstance(edge_index, Tensor):
                num_nodes = x_l.size(0)
                if x_r is not None:
                    num_nodes = min(num_nodes, x_r.size(0))
                edge_index, edge_attr = remove_self_loops(
                    edge_index, edge_attr)
                edge_index, edge_attr = add_self_loops(
                    edge_index, edge_attr, fill_value=self.fill_value,
                    num_nodes=num_nodes)
            elif isinstance(edge_index, SparseTensor):
                if self.edge_dim is None:
                    edge_index = set_diag(edge_index)
                else:
                    raise NotImplementedError(
                        "The usage of 'edge_attr' and 'add_self_loops' "
                        "simultaneously is currently not yet supported for "
                        "'edge_index' in a 'SparseTensor' form")

        # propagate_type: (x: PairTensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=(x_l, x_r), edge_attr=edge_attr,
                             size=None, n_nodes = x.size()[0])

        alpha = self._alpha
        self._alpha = None

        out = out.view(-1, C)
        out = self.nn(out)
        out = out.view(-1, H, self.out_channels)

        if self.concat:
            out = out.view(-1, H * self.out_channels)
        else:
            out = out.mean(dim=1)

        if isinstance(return_attention_weights, bool):
            assert alpha is not None
            if isinstance(edge_index, Tensor):
                return out, (edge_index, alpha)
            elif isinstance(edge_index, SparseTensor):
                return out, edge_index.set_value(alpha, layout='coo')
        else:
            return out


    def message(self, x_j: Tensor, x_i: Tensor, edge_attr: OptTensor,
                index: Tensor, ptr: OptTensor,
                size_i: Optional[int], n_nodes: int) -> Tensor:
        x = x_i + x_j
        if edge_attr is not None:
            if edge_attr.dim() == 1:
                edge_attr = edge_attr.view(-1, 1)
            assert self.lin_edge is not None
            edge_attr = self.lin_edge(edge_attr)
            edge_attr = edge_attr.view(-1, self.heads, self.out_channels)
            x = x + edge_attr

        x = F.leaky_relu(x, self.negative_slope)
        alpha = (x * self.att).sum(dim=-1)
        alpha = softmax(alpha, index, ptr, size_i)
        self._alpha = alpha
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)

        l = lambda x: 1 if x < alpha.size()[0] - n_nodes else 1 + self.eps
        m = torch.tensor([l(i) for i in range(alpha.size()[0])])
        m = torch.transpose(m.repeat(self.heads,1),0,1)
        alpha = m * alpha
        return x_j * alpha.unsqueeze(-1)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.in_channels}, '
                f'{self.out_channels}, heads={self.heads})')


## Models


#### GCN Module


In [ ]:
class GCN(nn.Module):
  def __init__(self,
               input_dim: int,
               hid_dim: int,
               output_dim: int,
               n_layers: int,
               dropout_ratio: float = 0.3):
    super(GCN, self).__init__()
    """
    Args:
      input_dim: input feature dimension
      hid_dim: hidden feature dimension
      output_dim: number of target classes
      n_layers: number of layers
      dropout_ratio: dropout_ratio
    """

    indim = lambda x: input_dim if x == 0 else hid_dim

    self.dropout_ratio = dropout_ratio
    self.n_layers = n_layers
    
    assert n_layers > 0

    self.layers = nn.Sequential(*[GCNConv(indim(i), hid_dim) for i in range(n_layers)])
    self.lin1 = Linear(hid_dim, hid_dim)
    self.lin2 = Linear(hid_dim, output_dim)

  def forward(self, x, edge_index, batch):
    # Generate Node Embeddings
    for layer in self.layers[:-2]:
      x = layer(x, edge_index)
      x = F.relu(x)
    h = self.layers[-1](x, edge_index)

    # Graph-level readout
    hG = global_mean_pool(h, batch)

    # Classification
    h = F.dropout(hG, p = self.dropout_ratio, training = self.training)
    h = self.lin1(h)
    h = h.relu()
    h = F.dropout(h, p = self.dropout_ratio, training = self.training)
    h = self.lin2(h)

    return hG, F.log_softmax(h, dim = 1)

  def reset_parameters(self):
    for layer in self.layers:
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
    self.lin1.reset_parameters()
    self.lin2.reset_parameters()
      


#### GIN Module


In [ ]:
class GIN(nn.Module):
  def __init__(self,
                input_dim: int,
                hid_dim: int,
                output_dim: int,
                n_layers: int,
                dropout_ratio: float = 0.3):
    super(GIN, self).__init__()
    """
    Args:
      input_dim: input feature dimension
      hid_dim: hidden feature dimension
      output_dim: number of target classes
      n_layers: number of layers
      dropout_ratio: dropout_ratio
    """

    indim = lambda x: input_dim if x == 0 else hid_dim
    
    self.dropout_ratio = dropout_ratio
    self.n_layers = n_layers
    
    assert n_layers > 0

    self.layers = nn.Sequential(*[GINConv(nn.Sequential(Linear(indim(i), hid_dim), 
                                                        nn.BatchNorm1d(hid_dim),
                                                        nn.ReLU(),
                                                        Linear(hid_dim, hid_dim),
                                                        nn.ReLU())) for i in range(n_layers)])
    self.lin1 = Linear(hid_dim * n_layers, hid_dim * n_layers)
    self.lin2 = Linear(hid_dim * n_layers, output_dim)

  def forward(self, x, edge_index, batch):
    embeddings = []
    for layer in self.layers:
      x = layer(x, edge_index)
      embeddings.append(x)
    
    # Graph-level read-out
    e = []
    for h in embeddings:
      h = global_add_pool(h, batch)
      e.append(h)
    embeddings = e

    h = torch.cat(embeddings, dim = 1)
    
    h = self.lin1(h)
    h = h.relu()
    h = F.dropout(h, p = self.dropout_ratio, training = self.training)
    h = self.lin2(h)

    return h, F.log_softmax(h, dim = 1)


  def reset_parameters(self):
    for layer in self.layers:
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
    self.lin1.reset_parameters()
    self.lin2.reset_parameters()


#### GAT Module


In [ ]:
class GAT(nn.Module):
  def __init__(self,
               input_dim: int,
               hid_dim: int,
               output_dim: int,
               n_layers: int,
               n_heads: int = 1,
               dropout_ratio: float = 0.3):
    super(GAT, self).__init__()
    """
    Args:
      input_dim: input feature dimension
      hid_dim: hidden feature dimension
      output_dim: number of target classes
      n_layers: number of layers
      n_heads: number of attention heads
      dropout_ratio: dropout_ratio
    """

    indim = lambda x: input_dim if x == 0 else hid_dim * n_heads

    self.dropout_ratio = dropout_ratio
    self.n_layers = n_layers
    
    assert n_layers > 0

    self.layers = nn.Sequential(*[GATConv(indim(i), hid_dim, heads = n_heads) for i in range(n_layers)])
    self.lin1 = Linear(hid_dim * n_heads, hid_dim * n_heads)
    self.lin2 = Linear(hid_dim * n_heads, output_dim)

  def forward(self, x, edge_index, batch):
    # Generate Node Embeddings
    for layer in self.layers[:-2]:
      x = layer(x, edge_index)
      x = F.elu(x)
      x = F.dropout(x, self.dropout_ratio, training = self.training)
    h = self.layers[-1](x, edge_index)

    # Graph-level readout
    hG = global_mean_pool(h, batch)

    # Classification
    h = F.dropout(hG, p = self.dropout_ratio, training = self.training)
    h = self.lin1(h)
    h = h.relu()
    h = F.dropout(h, p = self.dropout_ratio, training = self.training)
    h = self.lin2(h)

    return hG, F.log_softmax(h, dim = 1)

  def reset_parameters(self):
    for layer in self.layers:
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
    self.lin1.reset_parameters()
    self.lin2.reset_parameters()
  


#### AGIN Module

In [ ]:
class AGIN(nn.Module):
  def __init__(self,
                input_dim: int,
                hid_dim: int,
                output_dim: int,
                n_layers: int,
                n_heads: int,
                dropout_ratio: float = 0.3):
    super(AGIN, self).__init__()
    """
    Args:
      input_dim: input feature dimension
      hid_dim: hidden feature dimension
      output_dim: number of target classes
      n_layers: number of layers
      n_heads: number of attention heads
      dropout_ratio: dropout_ratio
    """
    
    indim = lambda x: input_dim if x == 0 else hid_dim * n_heads
    
    self.dropout_ratio = dropout_ratio
    self.n_layers = n_layers
    self.hid_dim = hid_dim
    self.n_heads = n_heads
    
    assert n_layers > 0

    self.layers = nn.Sequential(*[AGINConv(indim(i), 
                                           hid_dim, 
                                           nn.Sequential(Linear(indim(i), hid_dim), 
                                                        nn.BatchNorm1d(hid_dim),
                                                        nn.ReLU(),
                                                        Linear(hid_dim, hid_dim),
                                                        nn.ReLU()),
                                           heads = n_heads,
) for i in range(n_layers)])
    self.lin1 = Linear(hid_dim * n_heads * n_layers, hid_dim * n_heads * n_layers)
    self.lin2 = Linear(hid_dim * n_heads * n_layers, output_dim)

  def forward(self, x, edge_index, batch):
    embeddings = []
    for layer in self.layers:
      x = layer(x, edge_index)
      embeddings.append(x)
    
    # Graph-level read-out
    e = []
    for h in embeddings:
      h = global_add_pool(h, batch)
      e.append(h)
    embeddings = e

    h = torch.cat(embeddings, dim = 1)
    
    h = self.lin1(h)
    h = h.relu()
    h = F.dropout(h, p = self.dropout_ratio, training = self.training)
    h = self.lin2(h)

    return h, F.log_softmax(h, dim = 1)

  def reset_parameters(self):
    for layer in self.layers:
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
    self.lin1.reset_parameters()
    self.lin2.reset_parameters()


#### Instantiation

In [ ]:
agin = AGIN(dataset.num_features,
          8,
          dataset.num_classes,
          3,
          8)
gat = GAT(dataset.num_features,
          8,
          dataset.num_classes,
          3,
          8)

gcn = GCN(dataset.num_features,
          64,
          dataset.num_classes,
          3)

gin = GIN(dataset.num_features,
          64,
          dataset.num_classes,
          3)

## Training

In [ ]:
def train(model, train_loader, val_loader= val_loader, test_loader= test_loader, lr=0.01, weight_decay= 5e-4, epochs= 100, patience=5):
    criterion = torch.nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                      lr=lr,
                                      weight_decay=weight_decay)
    epochs = epochs

    model.train()
    print(model)
    c = 0 # Patience Counter
    previous = 0
    for epoch in range(epochs+1):
        total_loss = 0
        acc = 0
        val_loss = 0
        val_acc = 0

        # Train on batches
        for data in train_loader:
          optimizer.zero_grad()
          _, out = model(data.x, data.edge_index, data.batch)
          loss = criterion(out, data.y)
          total_loss += loss / len(train_loader)
          acc += accuracy(out.argmax(dim=1), data.y) / len(train_loader)
          loss.backward()
          optimizer.step()

          # Validation
          val_loss, val_acc = test(model, val_loader)
        
        # Early Stop
        if (acc < previous):
            c += 1 
            if (c > patience):
              break
        else:
            c = 0
        previous = acc
        
        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {total_loss:.2f} '
              f'| Train Acc: {acc*100:>5.2f}% '
              f'| Val Loss: {val_loss:.2f} '
              f'| Val Acc: {val_acc*100:.2f}%')
          
    test_loss, test_acc = test(model, test_loader)
    print(f'Test Loss: {test_loss:.2f} | Test Acc: {test_acc*100:.2f}%')
    
    return model

@torch.no_grad()
def test(model, loader):
    criterion = torch.nn.NLLLoss()
    model.eval()
    loss = 0
    acc = 0

    for data in loader:
        _, out = model(data.x, data.edge_index, data.batch)
        loss += criterion(out, data.y) / len(loader)
        acc += accuracy(out.argmax(dim=1), data.y) / len(loader)

    return loss, acc


def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

In [ ]:
agin = train(agin, train_loader, weight_decay=0.01)
gat = train(gat, train_loader, weight_decay=0.0005)
gcn = train(gcn, train_loader, weight_decay=0.0005)
gin = train(gin, train_loader, weight_decay=0.01)

## Cross Validation

In [ ]:
def cross_validate(model, dataset= dataset, num_splits=10, batch_size=32, epochs=200, weight_decay = 5e-4):
    """Use k-fold cross validation to evaluate a model on a dataset."""

    # Get the number of graphs and number of pairs
    size = len(dataset)

    # Arrays to store the train and test accuracies for each fold
    train_accuracies = np.zeros(num_splits)
    test_accuracies = np.zeros(num_splits)
    
    # Loop over the folds
    for fold in range(num_splits): 

        # Print a header to mark this fold
        print(f"Fold {fold+1}")
        print("============================")
        print()

        # Calculate the current fold segment indices
        index_min = int(size // num_splits * fold)
        index_max = int(size // num_splits * (fold + 1))

        # Split into train and test datasets

        train_dataset = (dataset[:index_min] 
                         + dataset[index_max:])
        test_dataset = dataset[index_min:index_max]

        # Turn these into torch_geometric dataloaders
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

        # Reset Parameters
        model.reset_parameters()

        # Train with these
        train(model, train_dataloader, test_dataloader, test_dataloader, weight_decay = weight_decay, epochs = epochs)

        # Record the test and train accuracies for the trained model
        _, train_accuracies[fold] = test(model, train_dataloader)
        _, test_accuracies[fold] = test(model, test_dataloader)
    
    # Print the Train and test accuracies for each fold
    print(f"{num_splits}-fold validation summary")
    print("============================")
    for fold in range(num_splits):
        print(f"Fold {fold+1}. Train: {train_accuracies[fold]:09.5%} "
              f"Test: {test_accuracies[fold]:09.5%}")


In [ ]:
cross_validate(agin)
cross_validate(gat)
cross_validate(gcn)
cross_validate(gin)